Part 6. MDP model solving

In [51]:
import mdptoolbox, mdptoolbox.example
import pandas as pd
import numpy as np
import random
from math import sqrt

In [52]:
#read probability matrix from part 5
ts_df = pd.read_csv("./dataset/generated_data/ts_df_9.csv")
ts_df = ts_df.iloc[:,1:]

In [53]:
ts_df = ts_df.rename(columns={'action': 'action_0', 'action_2': 'action'})
# ts_df.fillna(-1, inplace = True)
# ts_df['action'] = ts_df['action'].astype(int)
num_states = 9
num_actions = 3

In [54]:
def divide_CV(input, CV):
    data_list = list(input)
    random.shuffle(data_list)

    subset_size = len(data_list) // CV
    remainder = len(data_list) % CV

    subsets = []
    start_idx = 0
    for i in range(CV):
        end_idx = start_idx + subset_size + (1 if i < remainder else 0)
        subsets.append(data_list[start_idx:end_idx])
        start_idx = end_idx
    return subsets

#define the reward function
def Reward(a, b, mode, num_states, num_actions):
    R = np.zeros(num_states * num_actions).reshape(num_states,num_actions)
    if mode == "reward":
        R[0,:] += 1

    else:
        R[num_states - 1,:] -= 1

    
    R[:,1] -= a
    R[:,2] -= b

    return R

In [55]:
# divide the data into 1:9
# calculate P
# solve the MDP (for 2)
# 
CV = 10

patients = ts_df.iloc[:,0]
patients = set(patients)
subsets = divide_CV(patients, CV)

tot_sim, tot_const, tot_random, tot_sim_w, tot_real = [[],[]], [[],[]], [[],[]], [[],[]], [[],[]]
LEDD_total = [0 for _ in range(7)] #sim_r, sim_w, worst_p, worst_p, constant, random, actual 
LEDD_total = {"optimal":[[],[]],
                "random":[[],[]],
                "const":[[],[]],
                "worst":[[],[]],
                "real":[[],[]]}
tests = ["optimal", "random", "const", "worst", "real"]
actions_chosen = {
        t: [[],[]] for t in tests
        }
for i in range(CV+1):
    if i < CV:
        valid_idx = subsets[i]
        train_idx = []
        for j in range(CV):
            if i != j:
                train_idx += subsets[j]

        valid = ts_df[ts_df['PATNO'].isin(valid_idx)]
        train = ts_df[ts_df['PATNO'].isin(train_idx)]
    else:
        valid = ts_df

    #create probablity matrix P
    P_t = np.zeros(num_states*num_actions*num_states).reshape(num_actions, num_states, num_states)
    P_v = np.zeros(num_states*num_actions*num_states).reshape(num_actions, num_states, num_states)

    for a in range(num_actions):
        for s in range(num_states):
            for s_ in range(num_states - 1):
                criteria = ((train['cluster'] == s) & (train['action'] == a))
                criteria_2 = ((train['cluster'] == s) & (train['action'] == a) & (train['cluster_n'] == s_))
                P_t[a,s,s_] = criteria_2.sum()/criteria.sum()

            P_t[a,s,num_states - 1] = 1- sum(P_t[a,s,:])

    for a in range(num_actions):
        for s in range(num_states):
            for s_ in range(num_states - 1):
                criteria = ((ts_df['cluster'] == s) & (ts_df['action'] == a))
                criteria_2 = ((ts_df['cluster'] == s) & (ts_df['action'] == a) & (ts_df['cluster_n'] == s_))
                P_v[a,s,s_] = criteria_2.sum()/criteria.sum()
            
            P_v[a,s,num_states - 1] = 1 - sum(P_v[a,s,:])

    
    a, b = 0.01, 0.025
    R_r = Reward(a, b,"reward", num_states, num_actions)
    
    mdp_model_r = mdptoolbox.mdp.PolicyIteration(P_t, R_r, 0.99)
    mdp_model_r.run()
    # if i < CV:
    #     print(f'CV = {i+1} / Reward Model / {mdp_model_r.policy}' , end = "")
    # else:
    #     print(f'Full D / Reward Model / {mdp_model_r.policy}' , end = "")

    R_p = Reward(a, b,"penalty", num_states, num_actions)
    mdp_model_p = mdptoolbox.mdp.PolicyIteration(P_t, R_p, 0.99)
    mdp_model_p.run()
    #print(f' / Penalty Model / {mdp_model_p.policy}')

    R_r_w = - R_r
    mdp_model_r_w = mdptoolbox.mdp.PolicyIteration(P_t, R_r_w, 0.99)
    mdp_model_r_w.run()

    R_p_w = - R_p
    mdp_model_p_w = mdptoolbox.mdp.PolicyIteration(P_t, R_p_w, 0.99)
    mdp_model_p_w.run()

    #policy = {"r":mdp_model_r.policy, "p":mdp_model_p.policy, "r_w" : mdp_model_r_w.policy, "p_w" : mdp_model_r_w.policy}
    policy = {"r":mdp_model_r.policy, "p":mdp_model_p.policy, "r_w" : mdp_model_r_w.policy, "p_w" : mdp_model_r_w.policy}
    sim_reward, random_reward, const_reward, sim_reward_w, real_reward = [0,0], [0,0], [0,0], [0,0], [0,0]

    


    tot_n = 0
    for pat in valid_idx:
        pat_d = valid[valid['PATNO'] == pat]
        n = len(pat_d)
        init_LEDD = list(pat_d['LEDD'])[0]
        LEDD = {"optimal":[init_LEDD,init_LEDD],
                "random":[init_LEDD,init_LEDD],
                "const":[init_LEDD,init_LEDD],
                "worst":[init_LEDD,init_LEDD],
                "real":[0,0]} #sim_r, sim_p, worst_r, worst_p, constant, random
        LEDD_traj = {"optimal":[init_LEDD,init_LEDD],
                "random":[init_LEDD,init_LEDD],
                "const":[init_LEDD,init_LEDD],
                "worst":[init_LEDD,init_LEDD],
                "real":[0,0]}

        state = pat_d['cluster']
        state = [list(state)[0],list(state)[0]]
        
        LEDD_total["real"].append(pat_d["LEDD"].mean())

        #simul
        for t in tests:
            if state[0] == 0:
                if t == "optimal":
                    sim_reward[0] += 1
                elif t == "const":
                    const_reward[0] += 1
                elif t == "worst":
                    sim_reward_w[0] += 1
                elif t == "real":
                    real_reward[0] += 1
                else:
                    random_reward[0] += 1
            elif state[1] == 3:
                if t == "optimal":
                    sim_reward[1] -= 1
                elif t == "const":
                    const_reward[1] -= 1
                elif t == "worst":
                    sim_reward_w[1] -= 1
                elif t == "real":
                    real_reward[1] -= 1
                else:
                    random_reward[1] -= 1

            for nn in range(n-1):
                if t == "optimal":
                    action = [policy['r'][state[0]], policy['p'][state[1]]]
                elif t == "const":
                    action = [0,0]
                elif t == "worst":
                    action = [policy["r_w"][state[0]], policy["p_w"][state[1]]]
                elif t == "real":
                    action = [list(pat_d["action"])[nn],list(pat_d["action"])[nn]]
                else:
                    action = [random.choice(range(3)), random.choice(range(3))]    
                if action[0] == 1:
                    LEDD[t][0] += 35
                elif action[0] == 2:
                    LEDD[t][0] += 180
                else:
                    if t != "const":
                        LEDD[t][0] -= 28

                if action[1] == 1:
                    LEDD[t][1] += 35
                elif action[1] == 2:
                    LEDD[t][1] += 180
                else:
                    if t != "const":
                        LEDD[t][1] -= 28

                for i in range(2):
                    actions_chosen[t][i].append(action[i])
                
                
                LEDD_traj[t][0] += LEDD[t][0]
                LEDD_traj[t][1] += LEDD[t][1]

                state_ = [
                    random.choices(range(num_states), weights = P_v[int(action[0]),int(state[0]),:])[0],
                    random.choices(range(num_states), weights = P_v[int(action[1]),int(state[1]),:])[0]
                ]

                for k in range(2):
                    if action[k] == 1:
                        if t == "optimal":
                            sim_reward[k] -= 0.01
                        elif t == "const":
                            const_reward[k] -= 0.01
                        elif t == "worst":
                            sim_reward_w[k] -= 0.01
                        elif t == "real":
                            real_reward[k] -= 0.025
                        else:
                            random_reward[k] -= 0.01
                    elif action[k] == 2:
                        if t == "optimal":
                            sim_reward[k] -= 0.025
                        elif t == "const":
                            const_reward[k] -= 0.025
                        elif t == "worst":
                            sim_reward_w[k] -= 0.025
                        elif t == "real":
                            real_reward[k] -= 0.025
                        else:
                            random_reward[k] -= 0.025
                if state[0] == 0:
                    if t == "optimal":
                        sim_reward[0] += 1
                    elif t == "const":
                        const_reward[0] += 1
                    elif t == "worst":
                        sim_reward_w[0] += 1
                    elif t == "real":
                        real_reward[0] += 0.025
                    else:
                        random_reward[0] += 1
                elif state[1] == 3:
                    if t == "optimal":
                        sim_reward[1] -= 1
                    elif t == "const":
                        const_reward[1] -= 1
                    elif t == "worst":
                        sim_reward_w[1] -= 1
                    elif t == "real":
                        real_reward[1] -= 1
                    else:
                        random_reward[1] -= 1
                state = state_
            
            LEDD_traj[t][0] /= n
            LEDD_traj[t][1] /= n

            LEDD_total[t][0].append(LEDD_traj[t][0])
            LEDD_total[t][1].append(LEDD_traj[t][1])

        tot_n += n


    sim_reward = [s/tot_n for s in sim_reward]
    sim_reward_w = [s/tot_n for s in sim_reward_w]
    const_reward = [s/tot_n for s in const_reward]
    random_reward = [s/tot_n for s in random_reward]
    real_rewards = [s/tot_n for s in real_reward]

    for kk in range(2):
        tot_sim[kk].append(sim_reward[kk])
        tot_const[kk].append(const_reward[kk])
        tot_random[kk].append(random_reward[kk])
        tot_sim_w[kk].append(sim_reward_w[kk])
        tot_real[kk].append(real_rewards[kk])

tot1, tot2 = [], []
#Real total

#LEDD_total["real"] = np.mean(LEDD_total["real"])

for t in tests:
    LEDD_total[t][0] = np.mean(LEDD_total[t][0])
    LEDD_total[t][1] = np.mean(LEDD_total[t][1])
# print what kind of elements actions_chosen have copilot? 

for i in range(2):
    for t in tests:
        num_zeros = actions_chosen[t][i].count(0)
        num_ones = actions_chosen[t][i].count(1)
        num_twos = actions_chosen[t][i].count(2)
        print(f"`Actions chosen for {t}, {'reward mode' if i == 0 else 'penalty mode'} : 0s={num_zeros}, 1s={num_ones}, 2s={num_twos}`")
    print()

modes = ["reward", "penalty"]
for aa in range(2):
    print(modes[aa])
    print("opti",  "\t", np.round(np.mean(tot_sim[aa]),4), "\t", np.round(np.std(tot_sim[aa])/sqrt(CV),4))
    print("worst", "\t", np.round(np.mean(tot_sim_w[aa]),4), "\t", np.round(np.std(tot_sim_w[aa])/sqrt(CV),4))
    print("const",  "\t", np.round(np.mean(tot_const[aa]),4),  "\t", np.round(np.std(tot_const[aa])/sqrt(CV),4))
    print("random",  "\t", np.round(np.mean(tot_random[aa]),4), "\t", np.round(np.std(tot_random[aa])/sqrt(CV),4))
    print("real \t", np.round(np.mean(tot_real[aa]),4))


`Actions chosen for optimal, reward mode : 0s=3697, 1s=558, 2s=459`
`Actions chosen for random, reward mode : 0s=1574, 1s=1576, 2s=1564`
`Actions chosen for const, reward mode : 0s=4714, 1s=0, 2s=0`
`Actions chosen for worst, reward mode : 0s=0, 1s=672, 2s=4042`
`Actions chosen for real, reward mode : 0s=2927, 1s=1329, 2s=458`

`Actions chosen for optimal, penalty mode : 0s=2451, 1s=1699, 2s=564`
`Actions chosen for random, penalty mode : 0s=1594, 1s=1589, 2s=1531`
`Actions chosen for const, penalty mode : 0s=4714, 1s=0, 2s=0`
`Actions chosen for worst, penalty mode : 0s=0, 1s=678, 2s=4036`
`Actions chosen for real, penalty mode : 0s=2927, 1s=1329, 2s=458`

reward
opti 	 0.1494 	 0.0051
worst 	 0.0861 	 0.0051
const 	 0.1356 	 0.004
random 	 0.1274 	 0.0036
real 	 0.0049
penalty
opti 	 -0.1435 	 0.0045
worst 	 -0.129 	 0.0052
const 	 -0.138 	 0.006
random 	 -0.1203 	 0.0038
real 	 -0.1228
